In [49]:
import sys
!{sys.executable} -m pip install pygame

from itertools import cycle
import random

import pygame
from pygame.locals import *


FPS = 100
SCREENWIDTH  = 288
SCREENHEIGHT = 512
# amount by which base can maximum shift to left
PIPEGAPSIZE  = 100 # gap between upper and lower part of pipe
BASEY        = SCREENHEIGHT * 0.79
# image, sound and hitmask  dicts
IMAGES, SOUNDS, HITMASKS = {}, {}, {}

# list of all possible players (tuple of 3 positions of flap)
PLAYERS_LIST = (
    # red bird
    (
        'assets/sprites/redbird-upflap1.png',
        'assets/sprites/redbird-midflap1.png',
        'assets/sprites/redbird-downflap1.png',
    ),
    # blue bird
    (
        'assets/sprites/bluebird-upflap1.png',
        'assets/sprites/bluebird-midflap1.png',
        'assets/sprites/bluebird-downflap1.png',
    ),
    # yellow bird
    (
        'assets/sprites/yellowbird-upflap1.png',
        'assets/sprites/yellowbird-midflap1.png',
        'assets/sprites/yellowbird-downflap1.png',
    ),
)

# list of backgrounds
BACKGROUNDS_LIST = (
    'assets/sprites/background-day.png',
    'assets/sprites/background-night.png',
)

# list of pipes
PIPES_LIST = (
    'assets/sprites/pipe-green.png',
    'assets/sprites/pipe-red.png',
)


try:
    xrange
except NameError:
    xrange = range


def main(nextAgent):
    global SCREEN, FPSCLOCK
    pygame.init()
    FPSCLOCK = pygame.time.Clock()
    SCREEN = pygame.display.set_mode((SCREENWIDTH, SCREENHEIGHT))
    pygame.display.set_caption('Flappy Bird')

    # numbers sprites for score display
    IMAGES['numbers'] = (
        pygame.image.load('assets/sprites/0.png').convert_alpha(),
        pygame.image.load('assets/sprites/1.png').convert_alpha(),
        pygame.image.load('assets/sprites/2.png').convert_alpha(),
        pygame.image.load('assets/sprites/3.png').convert_alpha(),
        pygame.image.load('assets/sprites/4.png').convert_alpha(),
        pygame.image.load('assets/sprites/5.png').convert_alpha(),
        pygame.image.load('assets/sprites/6.png').convert_alpha(),
        pygame.image.load('assets/sprites/7.png').convert_alpha(),
        pygame.image.load('assets/sprites/8.png').convert_alpha(),
        pygame.image.load('assets/sprites/9.png').convert_alpha()
    )

    # game over sprite
    IMAGES['gameover'] = pygame.image.load('assets/sprites/gameover.png').convert_alpha()
    # message sprite for welcome screen
    IMAGES['message'] = pygame.image.load('assets/sprites/message.png').convert_alpha()
    # base (ground) sprite
    IMAGES['base'] = pygame.image.load('assets/sprites/base.png').convert_alpha()

    # sounds
    if 'win' in sys.platform:
        soundExt = '.wav'
    else:
        soundExt = '.ogg'

    SOUNDS['die']    = pygame.mixer.Sound('assets/audio/die' + soundExt)
    SOUNDS['hit']    = pygame.mixer.Sound('assets/audio/hit' + soundExt)
    SOUNDS['point']  = pygame.mixer.Sound('assets/audio/point' + soundExt)
    SOUNDS['swoosh'] = pygame.mixer.Sound('assets/audio/swoosh' + soundExt)
    SOUNDS['wing']   = pygame.mixer.Sound('assets/audio/wing' + soundExt)

    while True:
        # select random background sprites
        randBg = random.randint(0, len(BACKGROUNDS_LIST) - 1)
        IMAGES['background'] = pygame.image.load(BACKGROUNDS_LIST[randBg]).convert()

        # select random player sprites
        randPlayer = random.randint(0, len(PLAYERS_LIST) - 1)
        IMAGES['player'] = (
            pygame.image.load(PLAYERS_LIST[randPlayer][0]).convert_alpha(),
            pygame.image.load(PLAYERS_LIST[randPlayer][1]).convert_alpha(),
            pygame.image.load(PLAYERS_LIST[randPlayer][2]).convert_alpha(),
        )

        # select random pipe sprites
        pipeindex = random.randint(0, len(PIPES_LIST) - 1)
        IMAGES['pipe'] = (
            pygame.transform.rotate(
                pygame.image.load(PIPES_LIST[pipeindex]).convert_alpha(), 180),
            pygame.image.load(PIPES_LIST[pipeindex]).convert_alpha(),
        )

        # hismask for pipes
        HITMASKS['pipe'] = (
            getHitmask(IMAGES['pipe'][0]),
            getHitmask(IMAGES['pipe'][1]),
        )

        # hitmask for player
        HITMASKS['player'] = (
            getHitmask(IMAGES['player'][0]),
            getHitmask(IMAGES['player'][1]),
            getHitmask(IMAGES['player'][2]),
        )

        movementInfo = showWelcomeAnimation()
        crashInfo = mainGame(movementInfo, nextAgent)
        #showGameOverScreen(crashInfo)


def showWelcomeAnimation():
    """Shows welcome screen animation of flappy bird"""
    # index of player to blit on screen
    playerIndex = 0
    playerIndexGen = cycle([0, 1, 2, 1])
    # iterator used to change playerIndex after every 5th iteration
    loopIter = 0

    playerx = int(SCREENWIDTH * 0.2)
    playery = int((SCREENHEIGHT - IMAGES['player'][0].get_height()) / 2)

    messagex = int((SCREENWIDTH - IMAGES['message'].get_width()) / 2)
    messagey = int(SCREENHEIGHT * 0.12)

    basex = 0
    # amount by which base can maximum shift to left
    baseShift = IMAGES['base'].get_width() - IMAGES['background'].get_width()

    # player shm for up-down motion on welcome screen
    playerShmVals = {'val': 0, 'dir': 1}

    SOUNDS['wing'].play()
    return {
        'playery': playery + playerShmVals['val'],
        'basex': basex,
        'playerIndexGen': playerIndexGen,
    }


def mainGame(movementInfo, nextagent):
    SCREEN.fill([255,255,255])
    agent = [None]*10
    for i in range(10):
        agent[i] = nextagent()
    score = 10
    playerIndex = 0
    loopIter = 0
    playerIndexGen = movementInfo['playerIndexGen']
    playerx, playery = [int(SCREENWIDTH * 0.2)for _ in range(10)], [int(movementInfo['playery']) for _ in range(10)]
                       
    basex = movementInfo['basex']
    baseShift = IMAGES['base'].get_width() - IMAGES['background'].get_width()

    # get 2 new pipes to add to upperPipes lowerPipes list
    newPipe1 = getRandomPipe()
    newPipe2 = getRandomPipe()

    # list of upper pipes
    upperPipes = [
        {'x': SCREENWIDTH + 200, 'y': newPipe1[0]['y']},
        {'x': SCREENWIDTH + 200 + (SCREENWIDTH / 2), 'y': newPipe2[0]['y']},
    ]

    # list of lowerpipe
    lowerPipes = [
        {'x': SCREENWIDTH + 200, 'y': newPipe1[1]['y']},
        {'x': SCREENWIDTH + 200 + (SCREENWIDTH / 2), 'y': newPipe2[1]['y']},
    ]

    pipeVelX = -4

    # player velocity, max velocity, downward accleration, accleration on flap
    playerVelY    =  [-9]*10   # player's velocity along Y, default same as playerFlapped
    playerMaxVelY =  [10]*10   # max vel along Y, max descend speed
    playerMinVelY =  [-8]*10   # min vel along Y, max ascend speed
    playerAccY    =   [1]*10   # players downward accleration
    playerRot     =  [45]*10   # player's rotation
    playerVelRot  =   [3]*10   # angular speed
    playerRotThr  =  [20]*10   # rotation threshold
    playerFlapAcc =  [-9]*10   # players speed on flapping
    playerFlapped = [False]*10 # true when player flaps
    atLeastOneFlapped = False # True when at least one player flaps
    playerCrashed = [False]*10 # True when player has crashed

    while True:
        for event in pygame.event.get():
            if event.type == QUIT or (event.type == KEYDOWN and event.key == K_ESCAPE):
                pygame.quit()
                sys.exit()
        

        
        for birdname in range(10):
            if not playerCrashed[birdname]:
                i = 0
                while upperPipes[i]['x'] + IMAGES['pipe'][0].get_width() < playerx[birdname] :
                    i+= 1

                    
                    
                v = [playery[birdname],(upperPipes[i]['y'] + lowerPipes[i]['y'])/2]
                if agent[birdname].shouldJump(v) :
                    if playery[birdname] > -2 * IMAGES['player'][0].get_height():
                        playerVelY[birdname] = playerFlapAcc[birdname]
                        atLeastOneFlapped = True
                        

                # check for crash here
                crashTest = checkCrash({'x': playerx[birdname], 'y': playery[birdname], 'index': playerIndex},
                                       upperPipes, lowerPipes)
                if crashTest[0]:
                    playerCrashed[birdname] = True

                # check for score
                playerMidPos = playerx[birdname] + IMAGES['player'][0].get_width() / 2
                for pipe in upperPipes:
                    pipeMidPos = pipe['x'] + IMAGES['pipe'][0].get_width() / 2
                    if pipeMidPos <= playerMidPos < pipeMidPos + 4:
                        score += 1
                        SOUNDS['point'].play()

                # player's movement
                if playerVelY[birdname] < playerMaxVelY[birdname] and not playerFlapped[birdname]:
                    playerVelY[birdname] += playerAccY[birdname]
                if playerFlapped[birdname]:
                    playerFlapped[birdname] = False
                    
                playerHeight = IMAGES['player'][0].get_height()
                playery[birdname] += min(playerVelY[birdname], BASEY - playery[birdname] - playerHeight)


                playerSurface = IMAGES["player"][playerIndex]
                SCREEN.blit(playerSurface, (playerx[birdname], playery[birdname]))
        pygame.display.update()
            
        
        
        
        
        # wing sounds if at least one has flapped
        
        if atLeastOneFlapped:
            SOUNDS['wing'].play()
        
        # playerIndex basex change
        if (loopIter + 1) % 3 == 0:
            playerIndex = next(playerIndexGen)
            loopIter = (loopIter + 1) % 30
            basex = -((-basex + 100) % baseShift)     

        # add new pipe when first pipe is about to touch left of screen
        if 0 < upperPipes[0]['x'] < 5:
            newPipe = getRandomPipe()
            upperPipes.append(newPipe[0])
            lowerPipes.append(newPipe[1])
            
        # move pipes to left
        for uPipe, lPipe in zip(upperPipes, lowerPipes):
            uPipe['x'] += pipeVelX
            lPipe['x'] += pipeVelX

        # remove first pipe if its out of the screen
        if upperPipes[0]['x'] < -IMAGES['pipe'][0].get_width():
            upperPipes.pop(0)
            lowerPipes.pop(0)

        # draw sprites
        SCREEN.blit(IMAGES['background'], (0,0))

        for uPipe, lPipe in zip(upperPipes, lowerPipes):
            SCREEN.blit(IMAGES['pipe'][0], (uPipe['x'], uPipe['y']))
            SCREEN.blit(IMAGES['pipe'][1], (lPipe['x'], lPipe['y']))

        SCREEN.blit(IMAGES['base'], (basex, BASEY))
        # print score so player overlaps the score
        showScore(score)
        FPSCLOCK.tick(FPS)
        
        if playerCrashed == [True]*10:
            return





        
        
        
        
def playerShm(playerShm):
    """oscillates the value of playerShm['val'] between 8 and -8"""
    if abs(playerShm['val']) == 8:
        playerShm['dir'] *= -1

    if playerShm['dir'] == 1:
         playerShm['val'] += 1
    else:
        playerShm['val'] -= 1


def getRandomPipe():
    """returns a randomly generated pipe"""
    # y of gap between upper and lower pipe
    gapY = random.randrange(0, int(BASEY * 0.6 - PIPEGAPSIZE))
    gapY += int(BASEY * 0.2)
    pipeHeight = IMAGES['pipe'][0].get_height()
    pipeX = SCREENWIDTH + 10

    return [
        {'x': pipeX, 'y': gapY - pipeHeight},  # upper pipe
        {'x': pipeX, 'y': gapY + PIPEGAPSIZE}, # lower pipe
    ]


def showScore(score):
    """displays score in center of screen"""
    scoreDigits = [int(x) for x in list(str(score))]
    totalWidth = 0 # total width of all numbers to be printed

    for digit in scoreDigits:
        totalWidth += IMAGES['numbers'][digit].get_width()

    Xoffset = (SCREENWIDTH - totalWidth) / 2

    for digit in scoreDigits:
        SCREEN.blit(IMAGES['numbers'][digit], (Xoffset, SCREENHEIGHT * 0.1))
        Xoffset += IMAGES['numbers'][digit].get_width()


        
        
def checkCrash(player, upperPipes, lowerPipes):
    """returns True if player collders with base or pipes."""
    pi = player['index']
    player['w'] = IMAGES['player'][0].get_width()
    player['h'] = IMAGES['player'][0].get_height()

    # if player crashes into ground
    if player['y'] + player['h'] >= BASEY - 1:
        return [True, True]
    else:

        playerRect = pygame.Rect(player['x'], player['y'],
                      player['w'], player['h'])
        pipeW = IMAGES['pipe'][0].get_width()
        pipeH = IMAGES['pipe'][0].get_height()

        for uPipe, lPipe in zip(upperPipes, lowerPipes):
            # upper and lower pipe rects
            uPipeRect = pygame.Rect(uPipe['x'], uPipe['y'], pipeW, pipeH)
            lPipeRect = pygame.Rect(lPipe['x'], lPipe['y'], pipeW, pipeH)

            # player and upper/lower pipe hitmasks
            pHitMask = HITMASKS['player'][pi]
            uHitmask = HITMASKS['pipe'][0]
            lHitmask = HITMASKS['pipe'][1]

            # if bird collided with upipe or lpipe
            uCollide = pixelCollision(playerRect, uPipeRect, pHitMask, uHitmask)
            lCollide = pixelCollision(playerRect, lPipeRect, pHitMask, lHitmask)

            if uCollide or lCollide:
                return [True, False]

    return [False, False]

def pixelCollision(rect1, rect2, hitmask1, hitmask2):
    """Checks if two objects collide and not just their rects"""
    rect = rect1.clip(rect2)

    if rect.width == 0 or rect.height == 0:
        return False

    x1, y1 = rect.x - rect1.x, rect.y - rect1.y
    x2, y2 = rect.x - rect2.x, rect.y - rect2.y

    for x in xrange(rect.width):
        for y in xrange(rect.height):
            if hitmask1[x1+x][y1+y] and hitmask2[x2+x][y2+y]:
                return True
    return False

def getHitmask(image):
    """returns a hitmask using an image's alpha."""
    mask = []
    for x in xrange(image.get_width()):
        mask.append([])
        for y in xrange(image.get_height()):
            mask[x].append(bool(image.get_at((x,y))[3]))
    return mask


You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [22]:
import random
class Agent() :
    s = 11
    def shouldJump(self, v) :
        print(v)
        return random.random() < 0.1

In [23]:
def nextAgent() :
    return Agent()

In [50]:
main(nextAgent)

[244, -18.0]
[244, -18.0]
[244, -18.0]
[244, -18.0]
[244, -18.0]
[244, -18.0]
[244, -18.0]
[244, -18.0]
[244, -18.0]
[244, -18.0]
[236, -18.0]
[236, -18.0]
[236, -18.0]
[236, -18.0]
[236, -18.0]
[236, -18.0]
[236, -18.0]
[236, -18.0]
[236, -18.0]
[236, -18.0]
[228, -18.0]
[229, -18.0]
[229, -18.0]
[229, -18.0]
[229, -18.0]
[229, -18.0]
[229, -18.0]
[229, -18.0]
[229, -18.0]
[229, -18.0]
[221, -18.0]
[223, -18.0]
[223, -18.0]
[223, -18.0]
[223, -18.0]
[223, -18.0]
[223, -18.0]
[223, -18.0]
[223, -18.0]
[223, -18.0]
[215, -18.0]
[218, -18.0]
[218, -18.0]
[218, -18.0]
[218, -18.0]
[218, -18.0]
[218, -18.0]
[218, -18.0]
[218, -18.0]
[218, -18.0]
[210, -18.0]
[214, -18.0]
[214, -18.0]
[214, -18.0]
[214, -18.0]
[214, -18.0]
[214, -18.0]
[214, -18.0]
[214, -18.0]
[214, -18.0]
[206, -18.0]
[211, -18.0]
[211, -18.0]
[211, -18.0]
[211, -18.0]
[211, -18.0]
[211, -18.0]
[211, -18.0]
[211, -18.0]
[206, -18.0]
[203, -18.0]
[209, -18.0]
[209, -18.0]
[209, -18.0]
[203, -18.0]
[209, -18.0]
[209, -18.0]

[-51, -18.0]
[42, -18.0]
[155, -18.0]
[-50, -18.0]
[108, -18.0]
[-27, -18.0]
[-47, -18.0]
[36, -18.0]
[153, -18.0]
[-48, -18.0]
[100, -18.0]
[-31, -18.0]
[-42, -18.0]
[31, -18.0]
[152, -18.0]
[-45, -18.0]
[93, -18.0]
[-34, -18.0]
[-36, -18.0]
[27, -18.0]
[152, -18.0]
[-41, -18.0]
[87, -18.0]
[-36, -18.0]
[-29, -18.0]
[24, -18.0]
[153, -18.0]
[-36, -18.0]
[79, -18.0]
[-37, -18.0]
[-21, -18.0]
[22, -18.0]
[155, -18.0]
[-30, -18.0]
[72, -18.0]
[-37, -18.0]
[-12, -18.0]
[21, -18.0]
[147, -18.0]
[-23, -18.0]
[66, -18.0]
[-36, -18.0]
[-2, -18.0]
[21, -18.0]
[140, -18.0]
[-15, -18.0]
[61, -18.0]
[-34, -18.0]
[8, -18.0]
[22, -18.0]
[134, -18.0]
[-6, -18.0]
[57, -18.0]
[-31, -18.0]
[18, -18.0]
[24, -18.0]
[129, -18.0]
[4, -18.0]
[54, -18.0]
[-27, -18.0]
[28, -18.0]
[27, -18.0]
[121, -18.0]
[14, -18.0]
[52, -18.0]
[-22, -18.0]
[38, -18.0]
[31, -18.0]
[114, -18.0]
[24, -18.0]
[51, -18.0]
[-30, -18.0]
[48, -18.0]
[36, -18.0]
[108, -18.0]
[34, -18.0]
[51, -18.0]
[-37, -18.0]
[58, -18.0]
[28, -18.0]

[84, 106.0]
[325, 106.0]
[236, 106.0]
[-4, 106.0]
[139, 106.0]
[87, 106.0]
[61, 106.0]
[158, 106.0]
[83, 106.0]
[173, 106.0]
[85, 106.0]
[335, 106.0]
[233, 106.0]
[-8, 106.0]
[134, 106.0]
[84, 106.0]
[57, 106.0]
[150, 106.0]
[85, 106.0]
[166, 106.0]
[87, 106.0]
[345, 106.0]
[231, 106.0]
[-11, 106.0]
[130, 106.0]
[82, 106.0]
[54, 106.0]
[142, 106.0]
[88, 106.0]
[160, 106.0]
[90, 106.0]
[337, 106.0]
[223, 106.0]
[-13, 106.0]
[122, 106.0]
[81, 106.0]
[52, 106.0]
[135, 106.0]
[92, 106.0]
[155, 106.0]
[94, 106.0]
[330, 106.0]
[216, 106.0]
[-14, 106.0]
[115, 106.0]
[81, 106.0]
[51, 106.0]
[129, 106.0]
[97, 106.0]
[151, 106.0]
[99, 106.0]
[324, 106.0]
[210, 106.0]
[-14, 106.0]
[109, 106.0]
[82, 106.0]
[43, 106.0]
[124, 106.0]
[103, 106.0]
[148, 106.0]
[105, 106.0]
[319, 106.0]
[205, 106.0]
[-13, 106.0]
[104, 106.0]
[84, 106.0]
[35, 106.0]
[116, 106.0]
[110, 106.0]
[146, 106.0]
[112, 106.0]
[315, 106.0]
[201, 106.0]
[-11, 106.0]
[100, 106.0]
[76, 106.0]
[28, 106.0]
[108, 106.0]
[102, 106.0]
[1

SystemExit: 

C:\Users\Charlesmd50\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
random.random()

0.8421534952796442

In [4]:
import customFlappy
customFlappy.main(nextAgent)

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/Users/leoheidelberger/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


  Cache entry deserialization failed, entry ignored
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
